**Progetto di Introduzione all'Apprendimento Automatico di Catalini Yonas 
Ernesto, mat. 0000987636**

---



**Imports**

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model
import keras
from keras import activations
from keras import layers
from sklearn.model_selection import train_test_split

**Definizione generatore e funzione di accuracy**

In [2]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

In [3]:
def my_accuracy(true_maps: tf.Tensor, my_maps: tf.Tensor) -> float:
  return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(true_maps, axis=1), tf.argmax(my_maps, axis=1)), tf.float64))

**Generazione training e validation (test) set**

I dati generati vengono divisi tra training e validation set.

In [4]:
bsize = 2000
n_train = 4000000
n_test = 40000
g1,g2 = 10,10
gen = polar_generator(n_train+n_test,grid=(g1,g2),noise=0.002,flat=True)
(theta,rho),maps = next(gen)
inputs=np.array([i for i in zip(theta,rho)])

x_train,x_test,y_train,y_test = train_test_split(inputs,maps, test_size=n_test/(n_train+n_test),shuffle=True,random_state=1)

**Definizione del modello**

Il modello ha un un primo layer per ogni input, in modo tale da dare il giusto peso ad entrambi i valori e non soltanto alla loro combinazione. Questi due layer vengono uniti poi uniti e passiamo attraverso due layer densi prima dell'output.
Le dimensioni dei layer e le funzioni di attivazione sono state scelte attraverso vari test.

In [5]:
in1=Input(shape=(1))
in2=Input(shape=(1))

d1=Dense(2,activation=keras.activations.softsign)(in1)
d2=Dense(2,activation=keras.activations.softsign)(in2)
joint=tf.keras.layers.concatenate([d1,d2],axis=1)
elab_layer=Dense(20,activation=keras.activations.relu)(joint)
cond_layer=Dense(4,activation=keras.activations.relu)(elab_layer)
out=Dense(100,activation=keras.activations.softmax)(cond_layer)

net= Model([in1,in2],out)
net.summary()

net.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy',my_accuracy]
)



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 2)            4           ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 2)            4           ['input_2[0][0]']                
                                                                                              

In [6]:
#nn = keras.models.load_model("mr-roboto", custom_objects={"my_accuracy":my_accuracy})

**Training**

60 Epoche per questi dati sono state abbastanza, ma per sicurezza controlliamo la loss sul validation set generato inizialmente per fare early stopping.

In [7]:
history = net.fit(
  x=[x_train[:,0],x_train[:,1]],
  y=y_train,
  epochs=100,
  batch_size=bsize,
  validation_data=([x_test[:,0],x_test[:,1]], y_test),
  callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
)

Epoch 1/100
2000/2000 [==============================] - 21s 10ms/step - loss: 2.5411 - accuracy: 0.3560 - my_accuracy: 0.3560 - val_loss: 1.1949 - val_accuracy: 0.6759 - val_my_accuracy: 0.6759
Epoch 2/100
2000/2000 [==============================] - 19s 10ms/step - loss: 0.8068 - accuracy: 0.7730 - my_accuracy: 0.7730 - val_loss: 0.5902 - val_accuracy: 0.8199 - val_my_accuracy: 0.8199
Epoch 3/100
2000/2000 [==============================] - 19s 10ms/step - loss: 0.5048 - accuracy: 0.8476 - my_accuracy: 0.8476 - val_loss: 0.4428 - val_accuracy: 0.8663 - val_my_accuracy: 0.8663
Epoch 4/100
2000/2000 [==============================] - 19s 10ms/step - loss: 0.4034 - accuracy: 0.8796 - my_accuracy: 0.8796 - val_loss: 0.3708 - val_accuracy: 0.8869 - val_my_accuracy: 0.8869
Epoch 5/100
2000/2000 [==============================] - 20s 10ms/step - loss: 0.3425 - accuracy: 0.8993 - my_accuracy: 0.8993 - val_loss: 0.3159 - val_accuracy: 0.9073 - val_my_accuracy: 0.9073
Epoch 6/100
2000/2000 [==

**Testing**

Testo la rete 100 volte e stampo quanti di questi test risultano in accuracy inferiore al 95% e l'accuracy media.
Con questo modello l'accuracy media è sempre circa 96%, e i test che vanno sotto il 95% sono di solito intorno a 15/100.

In [8]:
gen = polar_generator(n_test,grid=(g1,g2),noise=0.002,flat=True)
accs = 0.0
lower = 0
tests = 100
for i in range(tests):
  (theta,rho),y = next(gen)
  x=np.array([i for i in zip(theta,rho)])
  score, _, acc = net.evaluate((x[:,0],x[:,1]), y, batch_size=bsize)
  accs += acc
  if acc < 0.95:
    lower += 1

print('Mean Accuracy: {:.1f}%'.format(accs/tests*100))
print('N. of tests with less accuracy than 95%: {}/{}'.format(lower, tests))

20/20 [==============================] - 0s 8ms/step - loss: 0.0881 - accuracy: 0.9715 - my_accuracy: 0.9715
Mean Accuracy: 95.9%
N. of tests with less accuracy than 95%: 18/100


In [9]:
#net.save('mr-roboto')

In [10]:
#!zip -r mr-roboto.zip mr-roboto